In [37]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [38]:
df = pd.read_csv(r"C:\Users\user\Downloads\Mental Health dataset1.csv")

In [39]:
df.head()

,Gender,Country,Occupation,SelfEmployed,FamilyHistory,Treatment,DaysIndoors,HabitsChange,MentalHealthHistory,IncreasingStress,MoodSwings
0,Female,UK,Others,No,Yes,No,15-30 days,No,Yes,Yes,High
1,Female,USA,Housewife,No,Yes,No,15-30 days,Maybe,Maybe,Yes,High
2,Female,Canada,Others,No,No,Yes,More than 2 months,Maybe,No,No,Medium
3,Female,Canada,Corporate,No,Yes,Yes,Go out Every day,No,Maybe,No,Medium
4,Female,UK,Corporate,No,No,Yes,More than 2 months,Yes,No,Maybe,Medium


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261328 entries, 0 to 261327
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Gender               261328 non-null  object
 1   Country              261328 non-null  object
 2   Occupation           261328 non-null  object
 3   SelfEmployed         256972 non-null  object
 4   FamilyHistory        261328 non-null  object
 5   Treatment            261328 non-null  object
 6   DaysIndoors          261328 non-null  object
 7   HabitsChange         261328 non-null  object
 8   MentalHealthHistory  261328 non-null  object
 9   IncreasingStress     261328 non-null  object
 10  MoodSwings           261328 non-null  object
dtypes: object(11)
memory usage: 21.9+ MB


In [41]:
df.describe()

,Gender,Country,Occupation,SelfEmployed,FamilyHistory,Treatment,DaysIndoors,HabitsChange,MentalHealthHistory,IncreasingStress,MoodSwings
count,261328,261328,261328,256972,261328,261328,261328,261328,261328,261328,261328
unique,2,35,5,2,2,2,5,3,3,3,3
top,Male,USA,Housewife,No,No,No,1-14 days,Yes,No,Maybe,Medium
freq,230614,154478,60081,230404,161981,132958,57598,97687,94027,90697,90215


In [42]:
df["SelfEmployed"].value_counts()

SelfEmployed
No     230404
Yes     26568
Name: count, dtype: int64

In [43]:
df["SelfEmployed"].fillna("No", inplace=True)

In [44]:
pd.crosstab(df["FamilyHistory"], df["IncreasingStress"])

IncreasingStress,Maybe,No,Yes
FamilyHistory,,,
No,56585,51071,54325
Yes,34112,31137,34098


In [45]:
x = df.drop("IncreasingStress", axis= 1)
y = df["IncreasingStress"]

In [46]:
from sklearn.model_selection import train_test_split
x_train , x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 42)

In [47]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

cat_col = x.columns

preprosessor = ColumnTransformer(transformers=[
    ("cats_col", OneHotEncoder(handle_unknown= "ignore"), cat_col)
])

In [48]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline([
    ("preprosessor", preprosessor),
    ("model", RandomForestClassifier(n_estimators= 100, max_depth= 10, class_weight= "balanced"))
])

In [49]:
pipe.fit(x_train, y_train)

,steps,"[('preprosessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cats_col', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [55]:
pipe.score(x_train, y_train), pipe.score(x_test, y_test)

(0.8452420812964575, 0.8426319213255271)

In [58]:
from sklearn.metrics import classification_report
y_pred = pipe.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       Maybe       0.88      0.82      0.85     18212
          No       0.82      0.87      0.84     16497
         Yes       0.83      0.84      0.83     17557

    accuracy                           0.84     52266
   macro avg       0.84      0.84      0.84     52266
weighted avg       0.84      0.84      0.84     52266



In [59]:
#y_pred = (pipe.predict_proba(x_test)[:,1] >= 0.5).astype (int)

In [60]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[14889,  1571,  1752],
       [  794, 14334,  1369],
       [ 1179,  1560, 14818]])

In [63]:
new_person = pd.DataFrame([{
    'Gender':'Male',
    'Country':'USA',
    'Occupation':'Student',
    'SelfEmployed':'No',
    'FamilyHistory':'Yes',
    'Treatment':'No',
    'DaysIndoors':'15-30 days',
    'HabitsChange':'Yes',
    'MentalHealthHistory':'Yes',
    'MoodSwings':'High'
}])

pipe.predict(new_person)


array(['Maybe'], dtype=object)

In [61]:
import pandas as pd

compare = pd.DataFrame({
    "Actual": y_test.values,
    "Predicted": y_pred
})

compare.head(50)

,Actual,Predicted
0,No,No
1,No,No
2,Maybe,Maybe
3,No,No
4,No,No
5,Yes,Yes
6,Yes,Yes
7,No,No
8,Yes,Yes
9,Maybe,Yes
